In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## 1. loaded data 

In [182]:
batch_path = os.getcwd()
path = os.path.join(batch_path, '..', 'data', 'spam.csv')
df = pd.read_csv(path, sep=',', encoding="latin-1")
df.head()
 

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [183]:
## drop the nan
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df[['v1', 'v2']]
df.columns= ['label', 'target']
df.head()


,label,target
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [184]:
df.isnull().sum()

label     0
target    0
dtype: int64

## 2. claning the data 

In [185]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


class TextPreprocessor:

    def __init__(self):
        nltk.download('punkt')
        nltk.download('stopwords')
        nltk.download('wordnet')

        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def preprocess(self, text):

        text = str(text).lower()
        text = text.translate(str.maketrans('', '', string.punctuation))

        tokens = word_tokenize(text)

        # Verb lemmatization (win, won, winning → win)
        tokens = [self.lemmatizer.lemmatize(word, pos='v') for word in tokens]

        tokens = [word for word in tokens if word not in self.stop_words]

        return " ".join(tokens)


## 3. spliting the data and vectorized from data

In [186]:
df.head()

,label,target
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [187]:
text_convert = TextPreprocessor()
X = df['target'].apply(text_convert.preprocess)

y = df['label']

y = y.map({'ham':0, 'spam':1})
len(X), len(y)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sumit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sumit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sumit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(5572, 5572)

In [188]:
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [189]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer()),
    ('smote', SMOTE()),  # SMOTE will act after TF-IDF dense conversion automatically
    ('clf', LogisticRegression())
])

pipeline.fit(X_train, y_train)
 

,steps,"[('tfidf', ...), ('smote', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [190]:
pipeline.score(X_test, y_test)

0.9766816143497757

In [191]:
pipeline.predict(X_test[:20]), y_test[:20]

(array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]),
 3245    0
 944     0
 1044    1
 2484    0
 812     1
 2973    0
 2991    0
 2942    0
 230     0
 1181    0
 1912    0
 1992    1
 5435    0
 4805    0
 401     0
 1859    0
 1344    0
 2952    1
 501     0
 3337    0
 Name: label, dtype: int64)

## 4. evlaution


In [192]:
from sklearn.metrics import confusion_matrix, f1_score

In [193]:
y_pred = pipeline.predict(X_test)
confusion_matrix(y_test, y_pred), f1_score(y_test, y_pred)

(array([[954,  11],
        [ 15, 135]]),
 0.9121621621621622)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "TEXT": ["Congratulations! You won a free ticket!", "Hey, are we meeting today?", 
             "Claim your prize now!", "Let's have lunch tomorrow."],
    "LABEL": ["spam", "ham", "spam", "ham"]
})
df['TEXT'] = df['TEXT'].apply(text_convert.preprocess)


In [195]:
df['TEXT']

0    congratulations win free ticket
1                     hey meet today
2                        claim prize
3                 let lunch tomorrow
Name: TEXT, dtype: object

In [196]:
Y_preds = pipeline.predict(df['TEXT'])

In [197]:
Y_preds

array([1, 0, 1, 0])

In [198]:
pred_prob = pipeline.predict_proba(df['TEXT'])
spam_pred = pred_prob[:,1] > 0.5

In [199]:
spam_pred

array([ True, False,  True, False])

## 5. Save the model

In [201]:
import joblib
import os


batch_model  = os.getcwd()
path_model = os.path.join(batch_model, '..', 'model.pkl')


 


batch_text = os.getcwd()
path_text = os.path.join(batch_text, '..', 'text_covert.pkl')

joblib.dump(pipeline, path_model)
joblib.dump(text_convert, path_text)


['c:\\Users\\sumit\\OneDrive\\Desktop\\project work\\Spam-SMS-Classifier-using-NLP\\models\\..\\text_covert.pkl']